# Lab 2: Multi-Layer Perceptrons, Backpropagation, and Evaluation

**Module:** Artificial Intelligence  
**Topic:** Neural Networks – Part 1

---

### Learning Objectives

By the end of this lab, you will be able to:

1. Implement a Multi-Layer Perceptron (MLP) with sigmoid activation from scratch
2. Train the MLP using the backpropagation algorithm
3. Apply data normalisation for neural network training
4. Classify the Iris dataset using your MLP implementation
5. Compute and interpret a confusion matrix
6. Evaluate model performance using cross-validation
7. Apply vector hashing for text classification

---

## Background

### From Perceptrons to MLPs

In Lab 1, we saw that single-layer perceptrons can only solve linearly separable problems. To solve more complex problems like XOR, we need **Multi-Layer Perceptrons (MLPs)** with hidden layers.

### The Sigmoid Activation Function

Unlike the step function, the **sigmoid function** is differentiable, which allows us to use gradient-based training:

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

Its derivative (needed for backpropagation) is:

$$\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))$$

### The Backpropagation Algorithm

Backpropagation trains MLPs by:

1. **Forward pass:** Compute outputs layer by layer
2. **Compute error:** Compare output to expected value
3. **Backward pass:** Propagate error gradients back through the network
4. **Update weights:** Adjust weights to reduce error

### Data Normalisation

Neural networks work best when input features are normalised to a consistent range (e.g., [0, 1] or [-1, 1]). This prevents features with larger values from dominating the learning process.

---

## Setup

Import the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully!")

---

## Exercise 1: Implement an MLP Class

Build an MLP with:

- Configurable number of input, hidden, and output neurons
- Sigmoid activation function
- Backpropagation training

In [ ]:
class MLP:
    """
    Multi-Layer Perceptron with one hidden layer.
    Uses sigmoid activation and backpropagation.
    """
    
    def __init__(self, n_inputs, n_hidden, n_outputs, learning_rate=0.1):
        """
        Initialise the MLP.
        
        Parameters:
        ----------
        n_inputs : int
            Number of input features
        n_hidden : int
            Number of hidden neurons
        n_outputs : int
            Number of output neurons
        learning_rate : float
            Learning rate for weight updates
        """
        self.learning_rate = learning_rate
        
        # Input to hidden layer weights (weights_ih) and biases (bias_h)
        self.weights_ih = np.random.uniform(-0.5, 0.5, (n_inputs, n_hidden))
        self.bias_h = np.random.uniform(-0.5, 0.5, n_hidden)
        
        # Hidden to output layer weights (weights_ho) and biases (bias_o)
        # TODO: add code here for weights_ho and bias_o
        ## Hint: see code for weights_ih and bias_h above
        
    def sigmoid(self, x):
        """Sigmoid activation function."""
        # TODO: Implement sigmoid
        ## Hint: see Lab 1
        # Clip to avoid overflow in exp
        
    def sigmoid_derivative(self, x):
        """Derivative of sigmoid: σ'(x) = σ(x) * (1 - σ(x))."""
        # TODO: Implement sigmoid derivative
        ## Hint: see Lab 1
        
    def forward(self, X):
        """
        Forward pass through the network.
        
        Parameters:
        ----------
        X : array-like, shape (n_samples, n_inputs)
            Input data
        
        Returns:
        --------
        array : Output activations
        """
        # TODO: code this entire method
        ## Hint: see Lab 1
        
    def backward(self, X, y):
        """
        Backward pass (backpropagation).
        
        Parameters:
        ----------
        X : array-like, shape (n_samples, n_inputs)
            Input data
        y : array-like, shape (n_samples, n_outputs)
            Target outputs
        """
        # TODO: code this entire method
        ## Hint: see Lab 1
        
    def train(self, X, y, epochs, verbose=True):
        """
        Train the network.
        
        Parameters:
        ----------
        X : array-like, shape (n_samples, n_inputs)
            Training inputs
        y : array-like, shape (n_samples, n_outputs)
            Training targets
        epochs : int
            Number of training epochs
        verbose : bool
            Print progress if True
        
        Returns:
        --------
        list : Training loss history
        """
        losses = []
        
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(X)
            
            # Compute loss (Mean Squared Error)
            loss = np.mean((y - output) ** 2)
            losses.append(loss)
            
            # Backward pass
            self.backward(X, y)
            
            if verbose and (epoch % 100 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}: Loss = {loss:.6f}")
        
        return losses
    
    def predict(self, X):
        """
        Make predictions.
        
        Returns the class with highest output activation.
        """
        output = self.forward(X)
        return np.argmax(output, axis=1)

print("MLP class defined successfully!")

---

## Exercise 2: Test the MLP on XOR

Before using the MLP on real data, verify it works on XOR (from Lab 1).

In [ ]:
# Define XOR data
# TODO: add code here to define XOR data
## Hint: see Lab 1

# Create and train MLP
np.random.seed(42)
mlp_xor = MLP(n_inputs=2, n_hidden=4, n_outputs=1, learning_rate=2.0)
losses_xor = mlp_xor.train(X_xor, y_xor, epochs=5000, verbose=False)

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses_xor)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('XOR Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

# Test predictions
print("\nXOR Predictions:")
print("-" * 50)

for i in range(len(X_xor)):
    output = mlp_xor.forward(X_xor[i:i+1])
    rounded = round(output[0][0])
    correct = "✓" if rounded == y_xor[i][0] else "✗"
    print(f"Input: {X_xor[i]} -> Output: {output[0][0]:.4f} -> Rounded: {rounded} (Expected: {y_xor[i][0]}) {correct}")

---

## Exercise 3: Load and Prepare the Iris Dataset

The Iris dataset contains 150 samples of iris flowers with:

- 4 features: sepal length, sepal width, petal length, petal width
- 3 classes: Setosa, Versicolor, Virginica

### Task 3.1: Load and Explore the Data

In [ ]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

print("Iris Dataset Overview")
print("=" * 50)
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature names: {iris.feature_names}")
print(f"Target names: {iris.target_names}")
print(f"\nClass distribution: {np.bincount(y)}")

### Task 3.2: Normalise the Data

Normalise features to [0, 1] range for better neural network performance.

In [ ]:
# Normalise features to [0, 1]
scaler = MinMaxScaler()
X_normalised = scaler.fit_transform(X)

print("Data Normalisation")
print("=" * 50)
print("Before normalisation:")
print(f"  Min: {X.min(axis=0)}")
print(f"  Max: {X.max(axis=0)}")
print("\nAfter normalisation:")
print(f"  Min: {X_normalised.min(axis=0)}")
print(f"  Max: {X_normalised.max(axis=0)}")

### Task 3.3: One-Hot Encode the Targets

For multi-class classification with neural networks, we convert class labels to one-hot encoding:

- Class 0 → [1, 0, 0]
- Class 1 → [0, 1, 0]
- Class 2 → [0, 0, 1]

In [ ]:
# One-hot encode targets
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

print("One-Hot Encoding")
print("=" * 50)
print(f"Original y[:5]: {y[:5]}")
print(f"\nOne-hot y[:5]:")
print(y_onehot[:5])
print(f"\nEncoding: 0 → [1,0,0], 1 → [0,1,0], 2 → [0,0,1]")

### Task 3.4: Split into Training and Test Sets

In [ ]:
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X_normalised, y_onehot, test_size=0.2, random_state=42, stratify=y
)

# Also keep original labels for evaluation
X_, y_train_labels, y_test_labels = train_test_split(
    X_normalised, y, test_size=0.2, random_state=42, stratify=y
)

print("Train/Test Split")
print("=" * 50)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

---